In [1]:
# First we brought in Pandas and our initial dataset.
import pandas as pd

data = pd.read_csv("Cape Cod Game Scores.csv")

In [2]:
data = data.iloc[:,1:]
data

,0,1,2,3,4,5,6,7,8,9
0,0,9,9,6,6,5,7,8,7,8
1,9,0,10,7,7,6,8,9,8,9
2,9,10,0,7,6,5,7,8,8,8
3,6,7,7,0,4,3,5,6,5,6
4,6,7,6,4,0,2,4,5,5,5
5,5,6,5,3,2,0,3,4,4,4
6,7,8,7,5,4,3,0,6,6,6
7,8,9,8,6,5,4,6,0,7,7
8,7,8,8,5,5,4,6,7,0,7
9,8,9,8,6,5,4,6,7,7,0


In [3]:
import gurobipy as gb 

model = gb.Model()

teams = 10

team_list = ["Bourne", "Cotuit", "Hyannis", "Falmouth", "Wareham", "Orleans", "Harwich", "Chatham", 
        "Brewster", "Yarmouth"]

#binary matrix for whether or not a matchup will happen
team_indices = [i for i in range(teams)]
west_indices = [i for i in range(0,5)]
east_indices = [i for i in range (5,10)]
no0_indices = [1,2,3,4,5,6,7,8,9]
no1_indices = [0,2,3,4,5,6,7,8,9]
no2_indices = [0,1,3,4,5,6,7,8,9]
no3_indices = [0,1,2,4,5,6,7,8,9]
no4_indices = [0,1,2,3,5,6,7,8,9]
no5_indices = [0,1,2,3,4,6,7,8,9]
no6_indices = [0,1,2,3,4,5,7,8,9]
no7_indices = [0,1,2,3,4,5,6,8,9]
no8_indices = [0,1,2,3,4,5,6,7,9]
no9_indices = [0,1,2,3,4,5,6,7,8]

friday_indices = [5,11,17,20,26,32,36]
saturday_indices = [0,6,12,18,21,27,37]

falmouth_mwf_indices = [2,4,5,8,11]
bourne_weekday_indices = [2,4,5,8,10]
brewster_weekday_indices = [2,3,4,5,8,9,10]

slots = 42
slot_indices = [n for n in range(slots)]

Set parameter Username
Academic license - for non-commercial use only - expires 2023-10-29


In [4]:
games = model.addVars(slot_indices, team_indices, team_indices, vtype = gb.GRB.BINARY)

for k in slot_indices:
    model.addConstr(gb.quicksum(games[k,i,j] for i in range(10) for j in range(10)) == 5)

# For each home team, they can only play a home game in game slot[n] 1 or 0 times 
for n in slot_indices:
    for i in team_indices:
        model.addConstr(
            gb.quicksum(games[n,i,j]
                        for j in team_indices) <= 1)
        
# Each team must play 21 total home games
for i in team_indices:
    model.addConstr(
    gb.quicksum(games[n,i,j]
               for n in slot_indices for j in team_indices) == 21)
    
# Each team can only have 1 or 0 away games in a slot   
for n in slot_indices:
    for j in team_indices:
        model.addConstr(
            gb.quicksum(games[n,i,j]
                        for i in team_indices) <= 1)

# Each team must play 21 total away games
for j in team_indices:
    model.addConstr(
    gb.quicksum(games[n,i,j]
               for n in slot_indices for i in team_indices) == 21)
    
#constraint to stop a team having a home and away game in the same slot        
for n in slot_indices:
    for i in team_indices:
        model.addConstr(
            gb.quicksum(games[n,i,j] for j in team_indices) + gb.quicksum(games[n,j,i]
                                                                         for j in team_indices) <= 1
        )

#team cannot play itself
for n in slot_indices:
    for i in team_indices:
        model.addConstr(
            games[n,i,i] == 0)


In [5]:
#must host every team in other conference 2 times (except cross division rival matchup)
for j in east_indices:
        model.addConstr(
            gb.quicksum(games[n,0,j] for n in slot_indices) == 2)
for j in east_indices:
        model.addConstr(
            gb.quicksum(games[n,1,j] for n in slot_indices) == 2)
for j in range (5,9):
        model.addConstr(
            gb.quicksum(games[n,2,j] for n in slot_indices) == 2)
for j in range (9,10):
        model.addConstr(
            gb.quicksum(games[n,2,j] for n in slot_indices) == 1)
for j in east_indices:
        model.addConstr(
            gb.quicksum(games[n,3,j] for n in slot_indices) == 2)
for j in east_indices:
        model.addConstr(
            gb.quicksum(games[n,4,j] for n in slot_indices) == 2)
for j in west_indices:
        model.addConstr(
            gb.quicksum(games[n,5,j] for n in slot_indices) == 2)
for j in west_indices:
        model.addConstr(
            gb.quicksum(games[n,6,j] for n in slot_indices) == 2)
for j in west_indices:
        model.addConstr(
            gb.quicksum(games[n,7,j] for n in slot_indices) == 2)
for j in west_indices:
        model.addConstr(
            gb.quicksum(games[n,8,j] for n in slot_indices) == 2)
for j in range (0,2):
        model.addConstr(
            gb.quicksum(games[n,9,j] for n in slot_indices) == 2)
for j in range (2,3):
        model.addConstr(
            gb.quicksum(games[n,9,j] for n in slot_indices) == 1)
for j in range (3,5):
        model.addConstr(
            gb.quicksum(games[n,9,j] for n in slot_indices) == 2)
        
#must host every team in other conference 1 time in first half (except cross division rival matchup)
for j in east_indices:
        model.addConstr(
            gb.quicksum(games[n,0,j] for n in range (20)) == 1)
for j in east_indices:
        model.addConstr(
            gb.quicksum(games[n,1,j] for n in range (20)) == 1)
for j in range (5,9):
        model.addConstr(
            gb.quicksum(games[n,2,j] for n in range (20)) == 1)
model.addConstr(
    gb.quicksum(games[n,2,9] for n in range (20)) == 0)
for j in east_indices:
        model.addConstr(
            gb.quicksum(games[n,3,j] for n in range (20)) == 1)
for j in east_indices:
        model.addConstr(
            gb.quicksum(games[n,4,j] for n in range (20)) == 1)
for j in west_indices:
        model.addConstr(
            gb.quicksum(games[n,5,j] for n in range (20)) == 1)
for j in west_indices:
        model.addConstr(
            gb.quicksum(games[n,6,j] for n in range (20)) == 1)
for j in west_indices:
        model.addConstr(
            gb.quicksum(games[n,7,j] for n in range (20)) == 1)
for j in west_indices:
        model.addConstr(
            gb.quicksum(games[n,8,j] for n in range (20)) == 1)
for j in range (0,2):
        model.addConstr(
            gb.quicksum(games[n,9,j] for n in range (20)) == 1)
model.addConstr(
    gb.quicksum(games[n,9,2] for n in range (20)) == 0)
for j in range (3,5):
        model.addConstr(
            gb.quicksum(games[n,9,j] for n in range (20)) == 1)

# teams must play 10 home games first half
model.addConstr(gb.quicksum(games[n,3,j] for j in team_indices for n in range (20)) == 10)
model.addConstr(gb.quicksum(games[n,4,j] for j in team_indices for n in range (20)) == 10)
model.addConstr(gb.quicksum(games[n,2,j] for j in team_indices for n in range (20)) == 10)
model.addConstr(gb.quicksum(games[n,7,j] for j in team_indices for n in range (20)) == 10)
model.addConstr(gb.quicksum(games[n,8,j] for j in team_indices for n in range (20)) == 10)
model.addConstr(gb.quicksum(games[n,1,j] for j in team_indices for n in range (20)) == 10)
model.addConstr(gb.quicksum(games[n,0,j] for j in team_indices for n in range (20)) == 10)
model.addConstr(gb.quicksum(games[n,9,j] for j in team_indices for n in range (20)) == 10)
model.addConstr(gb.quicksum(games[n,5,j] for j in team_indices for n in range (20)) == 10)
model.addConstr(gb.quicksum(games[n,6,j] for j in team_indices for n in range (20)) == 10)        

# hard coding in west division games
model.addConstr(gb.quicksum(games[n,0,1] for n in slot_indices) == 3)
model.addConstr(gb.quicksum(games[n,0,2] for n in slot_indices) == 3)
model.addConstr(gb.quicksum(games[n,0,3] for n in slot_indices) == 3)
model.addConstr(gb.quicksum(games[n,0,4] for n in slot_indices) == 2)
model.addConstr(gb.quicksum(games[n,1,0] for n in slot_indices) == 3)
model.addConstr(gb.quicksum(games[n,1,2] for n in slot_indices) == 3)
model.addConstr(gb.quicksum(games[n,1,3] for n in slot_indices) == 2)
model.addConstr(gb.quicksum(games[n,1,4] for n in slot_indices) == 3)
model.addConstr(gb.quicksum(games[n,2,0] for n in slot_indices) == 3)
model.addConstr(gb.quicksum(games[n,2,1] for n in slot_indices) == 3)
model.addConstr(gb.quicksum(games[n,2,3] for n in slot_indices) == 3)
model.addConstr(gb.quicksum(games[n,2,4] for n in slot_indices) == 3)
model.addConstr(gb.quicksum(games[n,3,0] for n in slot_indices) == 3)
model.addConstr(gb.quicksum(games[n,3,1] for n in slot_indices) == 2)
model.addConstr(gb.quicksum(games[n,3,2] for n in slot_indices) == 3)
model.addConstr(gb.quicksum(games[n,3,4] for n in slot_indices) == 3)
model.addConstr(gb.quicksum(games[n,4,0] for n in slot_indices) == 2)
model.addConstr(gb.quicksum(games[n,4,1] for n in slot_indices) == 3)
model.addConstr(gb.quicksum(games[n,4,2] for n in slot_indices) == 3)
model.addConstr(gb.quicksum(games[n,4,3] for n in slot_indices) == 3)

# hard coding in east division games
model.addConstr(gb.quicksum(games[n,5,6] for n in slot_indices) == 3)
model.addConstr(gb.quicksum(games[n,5,7] for n in slot_indices) == 2)
model.addConstr(gb.quicksum(games[n,5,8] for n in slot_indices) == 3)
model.addConstr(gb.quicksum(games[n,5,9] for n in slot_indices) == 3)
model.addConstr(gb.quicksum(games[n,6,5] for n in slot_indices) == 3)
model.addConstr(gb.quicksum(games[n,6,7] for n in slot_indices) == 3)
model.addConstr(gb.quicksum(games[n,6,8] for n in slot_indices) == 2)
model.addConstr(gb.quicksum(games[n,6,9] for n in slot_indices) == 3)
model.addConstr(gb.quicksum(games[n,7,5] for n in slot_indices) == 2)
model.addConstr(gb.quicksum(games[n,7,6] for n in slot_indices) == 3)
model.addConstr(gb.quicksum(games[n,7,8] for n in slot_indices) == 3)
model.addConstr(gb.quicksum(games[n,7,9] for n in slot_indices) == 3)
model.addConstr(gb.quicksum(games[n,8,5] for n in slot_indices) == 3)
model.addConstr(gb.quicksum(games[n,8,6] for n in slot_indices) == 2)
model.addConstr(gb.quicksum(games[n,8,7] for n in slot_indices) == 3)
model.addConstr(gb.quicksum(games[n,8,9] for n in slot_indices) == 3)
model.addConstr(gb.quicksum(games[n,9,5] for n in slot_indices) == 3)
model.addConstr(gb.quicksum(games[n,9,6] for n in slot_indices) == 3)
model.addConstr(gb.quicksum(games[n,9,7] for n in slot_indices) == 3)
model.addConstr(gb.quicksum(games[n,9,8] for n in slot_indices) == 3)

# Each west team must play their division teams 3 times (or 1 for rivals) each in first half
model.addConstr(gb.quicksum(games[n,0,1] for n in range (20)) + 
                gb.quicksum(games[n,1,0] for n in range (20)) == 3)
model.addConstr(gb.quicksum(games[n,0,2] for n in range (20)) + 
                gb.quicksum(games[n,2,0] for n in range (20)) == 3)
model.addConstr(gb.quicksum(games[n,0,3] for n in range (20)) + 
                gb.quicksum(games[n,3,0] for n in range (20)) == 3)
model.addConstr(gb.quicksum(games[n,0,4] for n in range (20)) + 
                gb.quicksum(games[n,4,0] for n in range (20)) == 1)
model.addConstr(gb.quicksum(games[n,1,2] for n in range (20)) + 
                gb.quicksum(games[n,2,1] for n in range (20)) == 3)
model.addConstr(gb.quicksum(games[n,1,3] for n in range (20)) + 
                gb.quicksum(games[n,3,1] for n in range (20)) == 1)
model.addConstr(gb.quicksum(games[n,1,4] for n in range (20)) + 
                gb.quicksum(games[n,4,1] for n in range (20)) == 3)
model.addConstr(gb.quicksum(games[n,2,3] for n in range (20)) + 
                gb.quicksum(games[n,3,2] for n in range (20)) == 3)
model.addConstr(gb.quicksum(games[n,2,4] for n in range (20)) + 
                gb.quicksum(games[n,4,2] for n in range (20)) == 3)
model.addConstr(gb.quicksum(games[n,3,4] for n in range (20)) + 
                gb.quicksum(games[n,4,3] for n in range (20)) == 3)

# Each east team must play their division teams 3 times (or 1 for rivals) each in first half
model.addConstr(gb.quicksum(games[n,5,6] for n in range (20)) + 
                gb.quicksum(games[n,6,5] for n in range (20)) == 3)
model.addConstr(gb.quicksum(games[n,5,7] for n in range (20)) + 
                gb.quicksum(games[n,7,5] for n in range (20)) == 1)
model.addConstr(gb.quicksum(games[n,5,8] for n in range (20)) + 
                gb.quicksum(games[n,8,5] for n in range (20)) == 3)
model.addConstr(gb.quicksum(games[n,5,9] for n in range (20)) + 
                gb.quicksum(games[n,9,5] for n in range (20)) == 3)
model.addConstr(gb.quicksum(games[n,6,7] for n in range (20)) + 
                gb.quicksum(games[n,7,6] for n in range (20)) == 3)
model.addConstr(gb.quicksum(games[n,6,8] for n in range (20)) + 
                gb.quicksum(games[n,8,6] for n in range (20)) == 1)
model.addConstr(gb.quicksum(games[n,6,9] for n in range (20)) + 
                gb.quicksum(games[n,9,6] for n in range (20)) == 3)
model.addConstr(gb.quicksum(games[n,7,8] for n in range (20)) + 
                gb.quicksum(games[n,8,7] for n in range (20)) == 3)
model.addConstr(gb.quicksum(games[n,7,9] for n in range (20)) + 
                gb.quicksum(games[n,9,7] for n in range (20)) == 3)
model.addConstr(gb.quicksum(games[n,8,9] for n in range (20)) + 
                gb.quicksum(games[n,9,8] for n in range (20)) == 3)


<gurobi.Constr *Awaiting Model Update*>

In [6]:
# Constraint to make a team not play more than 2 straight home games
for n in range(40):
    for i in team_indices:
        model.addConstr(
            gb.quicksum(games[n,i,j] for j in team_indices) + gb.quicksum(games[n+1,i,j] for j in team_indices)
            + gb.quicksum(games[n+2,i,j] for j in team_indices) <= 2)
        
# Constraint to make a team not play more than 2 straight away games
for n in range(40):
    for j in team_indices:
        model.addConstr(
            gb.quicksum(games[n,i,j] for i in team_indices) + gb.quicksum(games[n+1,i,j] for i in team_indices)
            + gb.quicksum(games[n+2,i,j] for i in team_indices) <= 2)



# Rivals may not play each other in slots 15-24
model.addConstr(gb.quicksum(games[n,0,4] for n in range (15,25)) + 
                gb.quicksum(games[n,4,0] for n in range (15,25)) == 0)
model.addConstr(gb.quicksum(games[n,3,1] for n in range (15,25)) + 
                gb.quicksum(games[n,1,3] for n in range (15,25)) == 0)
model.addConstr(gb.quicksum(games[n,2,9] for n in range (15,25)) + 
                gb.quicksum(games[n,9,2] for n in range (15,25)) == 0)
model.addConstr(gb.quicksum(games[n,7,5] for n in range (15,25)) + 
                gb.quicksum(games[n,5,7] for n in range (15,25)) == 0)
model.addConstr(gb.quicksum(games[n,8,6] for n in range (15,25)) + 
                gb.quicksum(games[n,6,8] for n in range (15,25)) == 0)
    
# Each team must play 4 division games in the last 5
model.addConstr(gb.quicksum(games[n,0,1] for n in range (37,42)) + 
                gb.quicksum(games[n,1,0] for n in range (37,42)) == 1)
model.addConstr(gb.quicksum(games[n,0,2] for n in range (37,42)) + 
                gb.quicksum(games[n,2,0] for n in range (37,42)) == 1)
model.addConstr(gb.quicksum(games[n,0,3] for n in range (37,42)) + 
                gb.quicksum(games[n,3,0] for n in range (37,42)) == 1)
model.addConstr(gb.quicksum(games[n,0,4] for n in range (37,42)) + 
                gb.quicksum(games[n,4,0] for n in range (37,42)) == 1)
model.addConstr(gb.quicksum(games[n,1,2] for n in range (37,42)) + 
                gb.quicksum(games[n,2,1] for n in range (37,42)) == 1)
model.addConstr(gb.quicksum(games[n,1,3] for n in range (37,42)) + 
                gb.quicksum(games[n,3,1] for n in range (37,42)) == 1)
model.addConstr(gb.quicksum(games[n,1,4] for n in range (37,42)) + 
                gb.quicksum(games[n,4,1] for n in range (37,42)) == 1)
model.addConstr(gb.quicksum(games[n,2,3] for n in range (37,42)) + 
                gb.quicksum(games[n,3,2] for n in range (37,42)) == 1)
model.addConstr(gb.quicksum(games[n,2,4] for n in range (37,42)) + 
                gb.quicksum(games[n,4,2] for n in range (37,42)) == 1)
model.addConstr(gb.quicksum(games[n,3,4] for n in range (37,42)) + 
                gb.quicksum(games[n,4,3] for n in range (37,42)) == 1)

# Each team must be home on either Sat or Sun
for i in team_indices:
    model.addConstr(
    gb.quicksum(games[0,i,j] for j in team_indices) +
    gb.quicksum(games[1,i,j] for j in team_indices) == 1)
for i in team_indices:
    model.addConstr(
    gb.quicksum(games[6,i,j] for j in team_indices) +
    gb.quicksum(games[7,i,j] for j in team_indices) == 1)
for i in team_indices:
    model.addConstr(
    gb.quicksum(games[12,i,j] for j in team_indices) +
    gb.quicksum(games[13,i,j] for j in team_indices) == 1)
for i in team_indices:
    model.addConstr(
    gb.quicksum(games[18,i,j] for j in team_indices) +
    gb.quicksum(games[19,i,j] for j in team_indices) == 1)
for i in team_indices:
    model.addConstr(
    gb.quicksum(games[21,i,j] for j in team_indices) +
    gb.quicksum(games[22,i,j] for j in team_indices) == 1)
for i in team_indices:
    model.addConstr(
    gb.quicksum(games[27,i,j] for j in team_indices) +
    gb.quicksum(games[28,i,j] for j in team_indices) == 1)
for i in team_indices:
    model.addConstr(
    gb.quicksum(games[37,i,j] for j in team_indices) +
    gb.quicksum(games[38,i,j] for j in team_indices) == 1)

# Each team needs 3 or 4 saturday home games
for i in team_indices:
    model.addConstr(
    gb.quicksum(games[n,i,j] for n in saturday_indices for j in team_indices) >= 3)
for i in team_indices:
    model.addConstr(
    gb.quicksum(games[n,i,j] for n in saturday_indices for j in team_indices) <= 4)
    
# Each team needs 3 or 4 friday home games
for i in team_indices:
    model.addConstr(
    gb.quicksum(games[n,i,j] for n in friday_indices for j in team_indices) >= 3)
for i in team_indices:
    model.addConstr(
    gb.quicksum(games[n,i,j] for n in friday_indices for j in team_indices) <= 4)
    

    
# Each team needs 3 home games and 3 away games in the last 6 games
for i in team_indices:
    model.addConstr(
        gb.quicksum(games[n,i,j]
                    for n in range (36,42) for j in team_indices) == 3)
    
# Each team needs 1 home game and 1 away game in the last 2 games
for i in team_indices:
    model.addConstr(
        gb.quicksum(games[n,i,j]
                    for n in range (40,42) for j in team_indices) == 1)

# Individual Requests
model.addConstr(games[39,4,0] == 1) # Wareham hosting Bourne
model.addConstr(games[35,2,9] == 1) # Hyannis hosting Yarmouth
model.addConstr(games[36,5,7] + games[41,5,7] == 1) # Orleans Chatham end of season
model.addConstr(games[36,7,5] + games [41,7,5] == 1) # Orleans Chatham end of season
model.addConstr(gb.quicksum(games[30,i,6] + games[35,i,6] for i in team_indices) == 2) # Harwich A July Wed's
model.addConstr(gb.quicksum(games[7,i,1] for i in team_indices) == 1) # Cotuit A Father's Day
model.addConstr(gb.quicksum(games[39,i,7] for i in team_indices) == 1) # Chatham A July 31
model.addConstr(games[19,6,7] + games [19,8,7] == 1) # Chatham A v Brewster or Harwich
model.addConstr(gb.quicksum(games[n,i,3] for n in falmouth_mwf_indices for i in team_indices) == 5)
# Falmouth A MWF in June

model.addConstr(gb.quicksum(games[n,i,0] for n in bourne_weekday_indices for i in team_indices) == 5)
# Bourne A weekdays (except Tuesday's) in June

model.addConstr(gb.quicksum(games[n,8,i] for n in brewster_weekday_indices for i in team_indices) <= 3)
# Brewster A 3+ weekdays in June

model.addConstr(games[0,0,8] + games[0,8,0] == 1) # Brewster Bourne Opening Day
model.addConstr(gb.quicksum(games[32,i,6] for i in team_indices) == 1) # Harwich A day before ASG
model.addConstr(games[36,6,8] + games[41,6,8] == 1) # Brewster Harwich end of season
model.addConstr(games[36,8,6] + games [41,8,6] == 1) # # Brewster Harwich end of season

# At least 4 games before repeating a matchup   
from itertools import product

for n in range(39):
    for i, j in product(team_indices, repeat=2):
        model.addConstr(gb.quicksum([games[n+k,i,j] + games[n+k,j,i] for k in range(4)]) <= 1)
        



In [7]:
#twoinfive_1 = 0
#
#for n in range (38):
 #   for i in team_indices:
  #      for j in team_indices:
   #         if sum(([games[n+k,i,j].X + games[n+k,j,i].X for k in range(5)])) == 2:
    #            twoinfive_1 = twoinfive_1+1

In [14]:
model.setObjective(
    gb.quicksum(games[n,0,j] + games[n+4,j,0]
                for n in range(38)
                for j in team_indices))
model.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6772 rows, 4200 columns and 96054 nonzeros
Model fingerprint: 0x43de2cd7
Variable types: 0 continuous, 4200 integer (4200 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]

Loaded MIP start from previous solve with objective 38

Presolve removed 4046 rows and 1079 columns
Presolve time: 0.77s
Presolved: 2726 rows, 3121 columns, 41651 nonzeros
Variable types: 0 continuous, 3121 integer (3121 binary)

Root relaxation: cutoff, 5030 iterations, 2.49 seconds (1.16 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0        38.00000   38.00000  0.00%     -    3s

Explored 1 nodes (5030 si

In [15]:
twoinfive_1 = 0

for n in range (38):
    for i in team_indices:
        for j in team_indices:
            if sum(([games[n+k,i,j].X + games[n+k,j,i].X for k in range(5)])) == 2:
                twoinfive_1 = twoinfive_1+1

print(twoinfive_1)

46


In [16]:
for i in team_indices:
    print(team_list[i], "Schedule:")
    print("")
    for n in range (20):
        for j in team_indices:
            if games[n,i,j].X == 1:
                print("slot",n,team_list[j])
            if games[n,j,i].X == 1:
                print("slot",n, "@",team_list[j])
    if i == 0:
        print("slot 20","@",team_list[4])
        print("slot 21",team_list[4])
    if i == 1:
        print("slot 20","@",team_list[3])
        print("slot 21",team_list[3])
    if i == 2:
        print("slot 20",team_list[9])
        print("slot 21","@",team_list[9])
    if i == 3:
        print("slot 20",team_list[1])
        print("slot 21","@",team_list[1])
    if i == 4:
        print("slot 20",team_list[0])
        print("slot 21","@",team_list[0])
    if i == 5:
        print("slot 20",team_list[7])
        print("slot 21","@",team_list[7])
    if i == 6:
        print("slot 20","@",team_list[8])
        print("slot 21",team_list[8])
    if i == 7:
        print("slot 20","@",team_list[5])
        print("slot 21",team_list[5])
    if i == 8:
        print("slot 20",team_list[6])
        print("slot 21","@",team_list[6])
    if i == 9:
        print("slot 20","@",team_list[2])
        print("slot 21",team_list[2])
    
    for n in range (20,42):
        for j in team_indices:
            if games[n,i,j].X == 1:
                print("slot",n+2,team_list[j])
            if games[n,j,i].X == 1:
                print("slot",n+2, "@",team_list[j])
                
    print("")

Bourne Schedule:

slot 0 @ Brewster
slot 1 Falmouth
slot 2 @ Orleans
slot 3 Cotuit
slot 4 @ Harwich
slot 5 @ Hyannis
slot 6 Yarmouth
slot 7 @ Wareham
slot 8 @ Chatham
slot 9 Cotuit
slot 10 @ Falmouth
slot 11 Hyannis
slot 12 Brewster
slot 13 @ Cotuit
slot 14 Harwich
slot 15 Chatham
slot 16 @ Yarmouth
slot 17 Hyannis
slot 18 Orleans
slot 19 @ Falmouth
slot 20 @ Wareham
slot 21 Wareham
slot 22 @ Yarmouth
slot 23 Harwich
slot 24 @ Brewster
slot 25 @ Hyannis
slot 26 Chatham
slot 27 Falmouth
slot 28 @ Orleans
slot 29 @ Hyannis
slot 30 Wareham
slot 31 @ Cotuit
slot 32 Orleans
slot 33 Brewster
slot 34 @ Falmouth
slot 35 Wareham
slot 36 @ Harwich
slot 37 Cotuit
slot 38 Yarmouth
slot 39 @ Chatham
slot 40 Falmouth
slot 41 @ Wareham
slot 42 @ Cotuit
slot 43 Hyannis

Cotuit Schedule:

slot 0 Chatham
slot 1 @ Yarmouth
slot 2 Wareham
slot 3 @ Bourne
slot 4 Brewster
slot 5 @ Orleans
slot 6 Hyannis
slot 7 @ Chatham
slot 8 Harwich
slot 9 @ Bourne
slot 10 @ Hyannis
slot 11 Falmouth
slot 12 @ Wareham
slot